In [1]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import torch
import torchvision.transforms.functional as TF
import tifffile
import yaml
from einops import rearrange


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from mushroom.data.multiplex import extract_ome_tiff, write_basic_ome_tiff, get_ome_tiff_channels, make_pseudo
from mushroom.data.visium import adata_from_visium
from mushroom.visualization.utils import display_sections
import mushroom.utils as utils

/data/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/data/estorrs/miniconda3/envs/mushroom/lib/python3.9/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [5]:
run_dir = '/data/estorrs/mushroom/data/projects/htan_talk'
mushroom_dir = os.path.join(run_dir, 'mushroom')
Path(mushroom_dir).mkdir(parents=True, exist_ok=True)

In [6]:
metadata = yaml.safe_load(open(os.path.join(run_dir, 'registered', 'metadata.yaml')))
metadata

{'S18-9906': {'data': {'he': {'S18-9906-U1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_he.tif',
    'S18-9906-U16': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s5_he.tif'},
   'multiplex': {'S18-9906-U10': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s4_multiplex.ome.tiff',
    'S18-9906-U18': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s7_multiplex.ome.tiff',
    'S18-9906-U25': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s9_multiplex.ome.tiff',
    'S18-9906-U3': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s2_multiplex.ome.tiff'},
   'visium': {'S18-9906-U1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_visium.h5ad',
    'S18-9906-U16': '/diskmnt/Projects/Users/estorrs/mushroom/data/projec

In [7]:
configs = {}
for case, data in metadata.items():
    config = [
        {
            'id': sid,
            'position': 0,
            'data': []
        }
        for sid in data['ids']
    ]
    
    for dtype, mapping in data['data'].items():
        for sample_id, filepath in mapping.items():
            idx = data['order'].index(sample_id)
            
            # convert from katmai to glacier
            filepath = filepath.replace('/diskmnt/Projects/Users/estorrs', '/data/estorrs')
            
            config[idx]['data'].append(
                {
                    'dtype': dtype,
                    'filepath': filepath,
                }
            )
    configs[case] = config
configs
    

{'S18-9906': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_he.tif'},
    {'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_visium.h5ad'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s1_xenium.h5ad'}]},
  {'id': 's2',
   'position': 0,
   'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s2_multiplex.ome.tiff'}]},
  {'id': 's3',
   'position': 0,
   'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s3_xenium.h5ad'}]},
  {'id': 's4',
   'position': 0,
   'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s4_multiplex.ome.tiff'}]},


In [8]:
import sys
yaml.safe_dump(configs, sys.stdout)

S18-9906:
- data:
  - dtype: he
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_he.tif
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_visium.h5ad
  id: s0
  position: 0
- data:
  - dtype: xenium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s1_xenium.h5ad
  id: s1
  position: 0
- data:
  - dtype: multiplex
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s2_multiplex.ome.tiff
  id: s2
  position: 0
- data:
  - dtype: xenium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s3_xenium.h5ad
  id: s3
  position: 0
- data:
  - dtype: multiplex
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s4_multiplex.ome.tiff
  id: s4
  position: 0
- data:
  - dtype: he
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s5_he.tif
  - dtype: visium
    filep

In [19]:
# # filling out position seperately and reading back in
# yaml.safe_dump(configs, open(os.path.join(mushroom_dir, 'configs_template.yaml'), 'w'))

In [9]:
configs = yaml.safe_load(open(os.path.join(mushroom_dir, 'configs_template.yaml')))
configs

{'S18-9906': [{'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_he.tif'},
    {'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_visium.h5ad'}],
   'id': 's0',
   'position': 0},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s1_xenium.h5ad'}],
   'id': 's1',
   'position': 5},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s2_multiplex.ome.tiff'}],
   'id': 's2',
   'position': 10},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s3_xenium.h5ad'}],
   'id': 's3',
   'position': 40},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s4_multiplex.ome.tiff'}],
   'id': 's4',
   'position': 45

In [10]:
for case, config in configs.items():
    directory = os.path.join(mushroom_dir, case)
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    yaml.safe_dump(config, open(os.path.join(directory, 'config.yaml'), 'w'))

In [11]:
mushroom_dir

'/data/estorrs/mushroom/data/projects/htan_talk/mushroom'